# Amazon Nova Video FMs Batch Inference with Bedrock
## Work with videos on a large scale
This notebook walks through the end-to-end process of running batch inference on a collection of videos stored in S3 using Amazon Nova Pro or Premium via Bedrock.

With [Batch Inference](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference.html), you can provide a set of prompts as a single input file and receive responses as a single output file, allowing you to get simultaneous large-scale predictions. The responses are processed and stored in your Amazon S3 bucket so you can access them at a later time. Amazon Bedrock offers support for Amazon Nova FMs for batch inference at a 50% lower price compared to on-demand inference pricing. Please refer to model list [here](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-supported.html).

## Introduction
**Amazon Bedrock** provides a unified API for invoking foundation models like Nova Lite, Nova Pro or Nova Premier as described [here](https://docs.aws.amazon.com/nova/latest/userguide/what-is-nova.html). When you have a **large set of videos** you want analyzed and summarized, captioned, or classified—you can use **batch inference**. This notebook shows you how to:

1. Discover all your MP4 videos in S3
2. Build a JSONL payload referencing them
3. Upload payload to S3
4. Kick off a Nova batch job
5. Poll for completion
6. Download the results locally and explore


## Table of Contents
1. [Prerequisites](#prerequisites)
2. [Install Dependencies](#install-dependencies)
3. [Configuration & Imports](#configuration--imports)
4. [Helper Functions](#helper-functions)
5. [List Videos in S3](#list-videos-in-s3)
6. [Build JSONL Payload](#build-jsonl-payload)
7. [Upload JSONL to S3](#upload-jsonl-to-s3)
8. [Invoke Batch Job](#invoke-batch-job)
9. [Poll Job Status](#poll-job-status)
10. [Download Results](#download-results)
11. [Conclusion](#conclusion)


## Prerequisites
Before you begin, ensure that you have the following prerequisites in place:
1. Updated boto3 to 1.35.1 or a greater version.
2. You must have permissions to invoke `CreateModelInvocationJob` and `GetModelInvocationJob` API. Refer to the documentation to learn about [required permissions for batch inference job](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-prereq.html#batch-inference-permissions).
3. Provide a S3 bucket with empty prefixes for `video/batch/input/` and `video/batch/output/`.
4. Bedrock Batch Inference requires a service role so that it can access and write to S3 on your behalf. You can create the service role manually [see here](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-iam-sr.html) or use the AWS Console workflow which can create one for you [here](https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/batch-inference/create). We also provide a quick way to create a service role in the code below. The role requires permissions to read and write data on Amazon S3 bucket (`GetObject`, `ListBucket`, `PutObject`).
5. This notebook was built using videos `.mp4` format.
6. Ensure that you are in a AWS region that is supported for Batch Inference. Refer [here](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-supported.html) for documentation.
7. The default maximum size of a single file (in GB) submitted for batch inference for Nova models is 1 GB. However, you can request an increase [here](https://us-east-1.console.aws.amazon.com/servicequotas/home/services/bedrock/quotas/L-68FC8D47) as needed.

### Upload videos to your Amazon S3 bucket
Upload the whole folder **academic_source** which you can find the the dataset folder to your bucket.
In case you want to use a different folder structure in Amazon S3, you have to change the variable **VIDEOS_SOURCE_PREFIX**.

In [3]:
import boto3 
print(boto3.__version__) 
# if not upgrade boto3 1.35.1 or greater version, uncomment below
# %pip install --upgrade pip
# %pip install boto3 --upgrade

1.38.3


In [4]:
import json
import logging
import re
import html
import os
import time
from IPython.display import display, Markdown, HTML
from urllib.parse import urlparse
from botocore.config import Config as BConfig

In [5]:
BUCKET_NAME          = 'batchnovabucket'
VIDEOS_SOURCE_PREFIX = 'video/batch/source/academic_source'
INPUT_PREFIX         = 'video/batch/input'
OUTPUT_PREFIX        = 'video/batch/output'
MODEL_ID             = 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0'
ROLE_ARN             = 'ROLE_ARN'
MAX_TOKENS           = 200
OUTPUT_FOLDER        = 'outputs'

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger(__name__)

# Initialize AWS clients
boto_cfg = BConfig(retries={'max_attempts':10,'mode':'standard'})
session = boto3.Session()
s3 = session.client('s3', config=boto_cfg)
bedrock = session.client('bedrock', config=boto_cfg)


In [5]:
## Helper Functions
from IPython.display import display, Markdown, HTML
import re
import html
def list_mp4s_in_s3(bucket, prefix):
    """
    Recursively list all .mp4 object keys under a given S3 prefix.
    """
    logger.info(f"Listing MP4s under s3://{bucket}/{prefix}/")
    token = None
    keys = []
    while True:
        params = {'Bucket': bucket, 'Prefix': prefix}
        if token:
            params['ContinuationToken'] = token
        resp = s3.list_objects_v2(**params)
        for obj in resp.get('Contents', []):
            k = obj['Key']
            if k.lower().endswith('.mp4'):
                keys.append(k)
                logger.info(f"  • {k}")
        if not resp.get('IsTruncated'):
            break
        token = resp.get('NextContinuationToken')
    return keys

def build_jsonl_from_s3_keys(keys, bucket):
    """
    Build a list of JSONL-ready records pointing to videos via s3Location.
    """
    records = []
    for idx, k in enumerate(keys):
        uri = f"s3://{bucket}/{k}"
        video_obj = {
            'video': {'format':'mp4', 'source':{'s3Location':{'uri':uri}}}
        }
        text_obj = {'text':'Please summarize this video in ~200 words.'}
        rec = {
            'recordId': f'video-{idx}',
            'modelInput':{
                'schemaVersion':'messages-v1',
                'messages':[{'role':'user','content':[text_obj, video_obj]}],
                'inferenceConfig':{'maxTokens':MAX_TOKENS}
            }
        }
        records.append(rec)
    return records


def pretty_llm_print(video_output, title=None):
    """Generates formatted output showing the prompt once and then video names and their summaries."""
    # Header styling
    header = ""
    if title:
        header = f"""<div style='border: 2px solid #000000; 
            padding: 10px; 
            border-radius: 5px; 
            max-width: fit-content; 
            margin: 0 auto; 
            text-align: center; 
            font-weight: bold;'>{title}</div>\n"""

    body_parts = [header]
    
    # Display the prompt (user's question) ONCE at the top
    if video_output:
        user_content = video_output[0]['modelInput']['messages'][0]['content']
        prompt_texts = [html.escape(c['text']).replace('\\n', '\n')
                        for c in user_content if 'text' in c]
        if prompt_texts:
            body_parts.append("\n**Prompt:**\n\n")
            body_parts.append("<div style='margin-bottom: 1em; font-style: italic;'>")
            body_parts.append("<br>".join(prompt_texts))
            body_parts.append("</div>")
            body_parts.append("\n\n---\n")  # Horizontal rule after prompt


    # Process each video entry
    for entry in video_output:
        # Extract video name from S3 URI
        video_uri = entry['modelInput']['messages'][0]['content'][1]['video']['source']['s3Location']['uri']
        video_name = video_uri.split('/')[-1]  # Get filename from URI
        
        # Add video name header
        body_parts.append(f"\n## 📹 {video_name}\n")
        
        # Process assistant summary
        assistant_content = entry['modelOutput']['output']['message']['content']
        for content in assistant_content:
            if 'text' in content:
                processed = process_content_string(content['text'])
                body_parts.append(processed)
        
        # Add separator between entries
        body_parts.append("\n\n---\n")

    # Final styling
    styled_markdown = f"""
<div style="border: 2px solid #FFC000; 
    padding: 10px; 
    border-radius: 5px; 
    max-width: 100%;">
{''.join(body_parts)}
</div>"""
    display(Markdown(styled_markdown))

def process_content_string(text):
    """Format thinking/answer blocks"""
    text = text.replace('\\n', '\n')
    
    answer_style = """<div style="background-color: #e8f5e9; 
        border-left: 4px solid #43a047; 
        padding: 10px; 
        margin: 10px 0; 
        border-radius: 4px;">
        <strong style="color: #43a047;">Summary</strong>
        <div style="margin-top: 8px; white-space: pre-wrap;">{}</div>
    </div>"""
    
    # Convert <answer> tags to summary blocks
    text = re.sub(r'<answer>(.*?)</answer>', 
                 lambda m: answer_style.format(m.group(1)), 
                 text, 
                 flags=re.DOTALL)
    
    return text

## List videos in Amazon S3
Use the helper function to find all MP4s under your source prefix in your Amazon S3 bucket.

In [6]:
mp4_keys = list_mp4s_in_s3(BUCKET_NAME, VIDEOS_SOURCE_PREFIX)
print(f"Found {len(mp4_keys)} videos.")
mp4_keys[:5]  # show first 5

2025-04-29 13:14:42,673 INFO Listing MP4s under s3://batchnovabucket/video/batch/source/academic_source/
2025-04-29 13:14:42,757 INFO   • video/batch/source/academic_source/Charades/0AGCS.mp4
2025-04-29 13:14:42,758 INFO   • video/batch/source/academic_source/Charades/0JJIY.mp4
2025-04-29 13:14:42,759 INFO   • video/batch/source/academic_source/Charades/13AUQ.mp4
2025-04-29 13:14:42,759 INFO   • video/batch/source/academic_source/Charades/13XM4.mp4
2025-04-29 13:14:42,760 INFO   • video/batch/source/academic_source/Charades/2IX2Z.mp4
2025-04-29 13:14:42,760 INFO   • video/batch/source/academic_source/Charades/2O5NR.mp4
2025-04-29 13:14:42,761 INFO   • video/batch/source/academic_source/Charades/38I4G.mp4
2025-04-29 13:14:42,762 INFO   • video/batch/source/academic_source/Charades/3RN5M.mp4
2025-04-29 13:14:42,762 INFO   • video/batch/source/academic_source/Charades/45BIP.mp4
2025-04-29 13:14:42,763 INFO   • video/batch/source/academic_source/Charades/51RLB.mp4
2025-04-29 13:14:42,763 I

Found 131 videos.


['video/batch/source/academic_source/Charades/0AGCS.mp4',
 'video/batch/source/academic_source/Charades/0JJIY.mp4',
 'video/batch/source/academic_source/Charades/13AUQ.mp4',
 'video/batch/source/academic_source/Charades/13XM4.mp4',
 'video/batch/source/academic_source/Charades/2IX2Z.mp4']

## Build JSONL Payload
Convert the list of keys into a newline-delimited JSON file for batch input. This is the required format for the batch job.

In [7]:
records = build_jsonl_from_s3_keys(mp4_keys, BUCKET_NAME)
input_key = f"{INPUT_PREFIX}/video_batch_{int(time.time())}.jsonl"
jsonl_str = '\n'.join(json.dumps(r) for r in records)
print(f"Generated {len(records)} records → {input_key}")
print(jsonl_str[:500], '...')

Generated 131 records → video/batch/input/video_batch_1745932487.jsonl
{"recordId": "video-0", "modelInput": {"schemaVersion": "messages-v1", "messages": [{"role": "user", "content": [{"text": "Please summarize this video in ~200 words."}, {"video": {"format": "mp4", "source": {"s3Location": {"uri": "s3://batchnovabucket/video/batch/source/academic_source/Charades/0AGCS.mp4"}}}}]}], "inferenceConfig": {"maxTokens": 200}}}
{"recordId": "video-1", "modelInput": {"schemaVersion": "messages-v1", "messages": [{"role": "user", "content": [{"text": "Please summarize this  ...


## Upload JSONL to S3
Put the JSONL file into your input prefix for the batch job.

In [10]:
s3.put_object(Bucket=BUCKET_NAME, Key=input_key, Body=jsonl_str.encode('utf-8'))
print(f"Uploaded JSONL to s3://{BUCKET_NAME}/{input_key}")

Uploaded JSONL to s3://batchnovabucket/video/batch/input/video_batch_1745932487.jsonl


## Invoke Batch Inference Job
Start the Nova Pro batch job. We point at the common `video/batch/` parent prefix.

In [11]:
resp = bedrock.create_model_invocation_job(
    jobName=f"batch-video-{int(time.time())}",
    modelId=MODEL_ID,
    inputDataConfig={ 's3InputDataConfig': {
        's3Uri': f"s3://{BUCKET_NAME}/video/batch/",
        's3InputFormat': 'JSONL'
    }},
    outputDataConfig={ 's3OutputDataConfig': {
        's3Uri': f"s3://{BUCKET_NAME}/{OUTPUT_PREFIX}/"
    }},
    roleArn=ROLE_ARN
)
job_arn = resp['jobArn']
print(f"Started job ARN: {job_arn}")

Started job ARN: arn:aws:bedrock:us-east-1:058264445765:model-invocation-job/xuant7wnfdzt


## Poll Job Status
Wait until the batch job completes.

In [12]:
while True:
    status_resp = bedrock.get_model_invocation_job(jobIdentifier=job_arn)
    status = status_resp['status']
    print('Status:', status)
    if status in ('Completed','Failed'):
        break
    time.sleep(10)
print('Final status:', status)

Status: Submitted
Status: Submitted
Status: Submitted
Status: Submitted
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Validating
Status: Scheduled
Status: Scheduled
Status: Scheduled
Status: Scheduled
Status: Scheduled
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
St

## Download Results
Fetch the generated JSONL output files to your local `./outputs/` folder.

In [13]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
out_uri = status_resp['outputDataConfig']['s3OutputDataConfig']['s3Uri']
parsed = urlparse(out_uri)
out_bucket, out_prefix = parsed.netloc, parsed.path.lstrip('/')
paginator = s3.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=out_bucket, Prefix=out_prefix):
    for obj in page.get('Contents', []):
        if not obj['Key'].lower().endswith('.jsonl.out'):
            continue
        dst = os.path.join(OUTPUT_FOLDER, os.path.basename(obj['Key']))
        s3.download_file(out_bucket, obj['Key'], dst)
        print('Downloaded →', dst)

Downloaded → outputs/video_batch_1745875178.jsonl.out
Downloaded → outputs/video_batch_1745908166.jsonl.out
Downloaded → outputs/video_batch_1745908205.jsonl.out
Downloaded → outputs/video_batch_1745875178.jsonl.out
Downloaded → outputs/video_batch_1745875178.jsonl.out
Downloaded → outputs/video_batch_1745908166.jsonl.out
Downloaded → outputs/video_batch_1745908205.jsonl.out
Downloaded → outputs/video_batch_1745931573.jsonl.out
Downloaded → outputs/video_batch_1745932487.jsonl.out
Downloaded → outputs/video_batch_1745875178.jsonl.out
Downloaded → outputs/video_batch_1745908166.jsonl.out
Downloaded → outputs/video_batch_1745908205.jsonl.out
Downloaded → outputs/video_batch_1745931573.jsonl.out
Downloaded → outputs/video_batch_1745932487.jsonl.out
Downloaded → outputs/video_batch_1745875178.jsonl.out
Downloaded → outputs/video_batch_1745908166.jsonl.out
Downloaded → outputs/video_batch_1745908205.jsonl.out
Downloaded → outputs/video_batch_1745931573.jsonl.out


In [15]:
# extract summaries from first output file after batch processing
output_path = "../batch/outputs"
output_files = os.listdir(output_path)
objects = []
with open(os.path.join(output_path, output_files[0]), 'r') as f:
    for line in f:
        obj = json.loads(line)
        objects.append(obj)

In [16]:
# the first 5 summaries will be displayed
first_five = objects[:5]
model_name = MODEL_ID.rpartition('/')[-1]
pretty_llm_print(first_five, title="Video summaries by batch processing with " + model_name)


<div style="border: 2px solid #FFC000; 
    padding: 10px; 
    border-radius: 5px; 
    max-width: 100%;">
<div style='border: 2px solid #000000; 
            padding: 10px; 
            border-radius: 5px; 
            max-width: fit-content; 
            margin: 0 auto; 
            text-align: center; 
            font-weight: bold;'>Video summaries by batch processing with amazon.nova-pro-v1:0</div>

**Prompt:**

<div style='margin-bottom: 1em; font-style: italic;'>Please summarize this video in ~200 words.</div>

---

## 📹 0AGCS.mp4
The video starts with a person standing in a room, holding a plaid blanket and looking down. The room has a white door, a wooden chair, and a red chair on the right side. There is a dog in the room, and a power outlet is mounted on the wall. The person then wraps the blanket around their body and continues to stand in the room. The person then proceeds to fold the blanket and places it on the wooden chair. The video ends with the person standing in the room, wearing a black jacket and gray pants.

---

## 📹 0JJIY.mp4
The video begins with a view of a room with a desk, shelves, and various items. A woman enters the room and begins to arrange items on the desk. She picks up a plastic bag and a jug, and then pours the contents of the jug into the bag. She then puts the bag down and picks up a frame, examining it before placing it on the desk. The woman continues to arrange items on the desk, moving things around and organizing them.

---

## 📹 13AUQ.mp4
Two boys are in the kitchen. One is standing at the counter and the other is standing next to him. The boy at the counter is eating something and the other boy is holding a plastic bag. There is a couch, a lamp, and a picture frame in the room.

---

## 📹 13XM4.mp4
A man is walking into a room with a container of fruit. He takes out a fruit and pours juice into a cup. He then walks to the other side of the room and holds the cup while talking.

---

## 📹 2IX2Z.mp4
The video depicts a man standing in a kitchen, holding a small object in his hand. He appears to be inspecting or examining the object closely. The kitchen is well-lit, and various kitchen items and appliances are visible in the background. The man's focus remains on the object throughout the video.

---

</div>

### Integrating with Existing Workflows

After retrieving the processed output data, you can integrate it into your existing workflows or analytics systems for further analysis or downstream processing. For example, you could:

- Store the summarized videos in a database for easy access and querying.
- Perform sentiment analysis or topic modeling on the summarized transcripts to gain additional insights.
- Categorize the summarizes into actionable business buckets.

The specific integration steps will depend on your existing workflows and systems, but the processed output data from the batch inference job can be easily incorporated into various data pipelines and analytics processes.

## Conclusion

The notebook covers the entire process, from data preparation and formatting to job submission, output retrieval, and integration with existing workflows. You can leverage the JSONL outputs for further analysis or visualization. Feel free to adapt and extend this notebook to suit your specific requirements, and explore other use cases where batch inference can be applied to optimize your interactions with foundation models at scale. 